In [2]:
import pandas as pd
data = pd.read_csv('/content/IMDB Dataset.csv')


In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
from tensorflow.keras import regularizers

from tensorflow.keras import layers
from tensorflow.keras import losses

from collections import Counter


import pandas as pd
import numpy as np

import sklearn


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


import seaborn as sns

import pydot


In [36]:
X = data.iloc[:, 0]
y = data.iloc[:, 1]

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)
y

array([1, 1, 1, ..., 1, 1, 1])

In [5]:
from bs4 import BeautifulSoup 
import re
import numpy as np
import nltk
from nltk.corpus import stopwords
import pandas as pd
nltk.download('punkt')
nltk.download('stopwords')


tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def review_wordlist(review, remove_stopwords=True):
    # 1. Removing html tags
    review_text = BeautifulSoup(review).get_text()
    # 2. Removing non-letter.
    review_text = re.sub("[^a-zA-Z]"," ",review_text)
    # 3. Converting to lower case and splitting
    words = review_text.lower().split()
    # 4. Optionally remove stopwords
    if remove_stopwords:
        stops = set(stopwords.words("english"))     
        words = [w for w in words if not w in stops]
    
    return(words)
# This function splits a review into sentences
def review_sentences(review, tokenizer, remove_stopwords=True):
    # 1. Using nltk tokenizer
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    # 2. Loop for each sentence
    for raw_sentence in raw_sentences:
        if len(raw_sentence)>0:
            t = review_wordlist(raw_sentence,remove_stopwords)
            t = " ".join(t)
            sentences.append(t)

    # This returns the list of lists
    return sentences
sentences = []
print("Parsing sentences from training set")
for review in X:
    sentences.append( " ".join(review_sentences(review, tokenizer)))


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sentences, y, test_size = 0.2)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Parsing sentences from training set


/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:273: UserWarning: "b'..'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


In [37]:
vocab_size = 5000 # make the top list of words (common words)
embedding_dim = 64
max_length = 100
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>' # OOV = Out of Vocabulary
training_portion = .8

In [38]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [39]:
train_sequences = tokenizer.texts_to_sequences(X_train)

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [40]:
validation_sequences = tokenizer.texts_to_sequences(X_test)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [41]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
training_label_seq  = le.fit_transform(y_train)
validation_label_seq  = le.transform(y_test)

In [11]:
training_label_seq

array([0, 1, 0, ..., 1, 1, 1])

In [53]:
max_features =50000
embedding_dim =64
sequence_length = 100

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(max_features +1, embedding_dim, input_length=sequence_length,\
                                    embeddings_regularizer = regularizers.l2(0.005))) 
model.add(tf.keras.layers.Dropout(0.04))

model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim,dropout=0.02, recurrent_dropout=0.2,return_sequences=True,\
                                                             kernel_regularizer=regularizers.l2(0.005),\
                                                             bias_regularizer=regularizers.l2(0.005))))
model.add(tf.keras.layers.Dropout(0.04))
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(512, activation='relu',\
                                kernel_regularizer=regularizers.l2(0.001),\
                                bias_regularizer=regularizers.l2(0.001),))
model.add(tf.keras.layers.Dropout(0.04))

model.add(tf.keras.layers.Dense(8, activation='relu',\
                                kernel_regularizer=regularizers.l2(0.001),\
                                bias_regularizer=regularizers.l2(0.001),))
model.add(tf.keras.layers.Dropout(0.04))


model.add(tf.keras.layers.Dense(1,activation='sigmoid'))
                               



model.summary()
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),optimizer=tf.keras.optimizers.Adam(lr=0.001, decay=1e-6),metrics=['accuracy'])




#model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)))
#model.add(tf.keras.layers.Dense(15, activation='sigmoid'))

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 100, 64)           3200064   
_________________________________________________________________
dropout_47 (Dropout)         (None, 100, 64)           0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 100, 128)          66048     
_________________________________________________________________
dropout_48 (Dropout)         (None, 100, 128)          0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 12800)             0         
_________________________________________________________________
dense_41 (Dense)             (None, 512)               6554112   
_________________________________________________________________
dropout_49 (Dropout)         (None, 512)             

In [54]:
num_epochs = 10
history = model.fit(train_padded, training_label_seq,  batch_size=128, epochs=num_epochs, validation_data=(validation_padded, validation_label_seq), verbose=2)


Epoch 1/10
140/140 - 106s - loss: 2.8403 - accuracy: 0.7687 - val_loss: 1.0556 - val_accuracy: 0.8243
Epoch 2/10
140/140 - 100s - loss: 0.9298 - accuracy: 0.8519 - val_loss: 0.8911 - val_accuracy: 0.8312
Epoch 3/10
140/140 - 106s - loss: 0.7782 - accuracy: 0.8623 - val_loss: 0.7797 - val_accuracy: 0.8294
Epoch 4/10
140/140 - 102s - loss: 0.6869 - accuracy: 0.8621 - val_loss: 0.6963 - val_accuracy: 0.8419
Epoch 5/10
140/140 - 98s - loss: 0.6062 - accuracy: 0.8705 - val_loss: 0.6629 - val_accuracy: 0.8272
Epoch 6/10
140/140 - 97s - loss: 0.5458 - accuracy: 0.8705 - val_loss: 0.5793 - val_accuracy: 0.8399
Epoch 7/10
140/140 - 96s - loss: 0.5015 - accuracy: 0.8762 - val_loss: 0.5440 - val_accuracy: 0.8417
Epoch 8/10
140/140 - 96s - loss: 0.4681 - accuracy: 0.8805 - val_loss: 0.5258 - val_accuracy: 0.8393
Epoch 9/10
140/140 - 94s - loss: 0.4373 - accuracy: 0.8826 - val_loss: 0.5057 - val_accuracy: 0.8357
Epoch 10/10
140/140 - 95s - loss: 0.4233 - accuracy: 0.8827 - val_loss: 0.5067 - val_ac